In [1]:
import requests
import pandas as pd
import numpy as np
import json
import os
from sklearn import preprocessing
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook

In [4]:
logo_urls = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215')[['shortname','logourl']].dropna(); logo_urls

,shortname,logourl
3,8seconds,https://ko.wikipedia.org/wiki/%EC%97%90%EC%9E%...
4,adidas,http://i0.wp.com/s3.store.hypebeast.com/media/...
5,adidasoriginal,http://i0.wp.com/s3.store.hypebeast.com/media/...
6,asics,http://i0.wp.com/s3.store.hypebeast.com/media/...
7,balenciaga,http://i0.wp.com/s3.store.hypebeast.com/media/...
8,bally,https://upload.wikimedia.org/wikipedia/en/thum...
9,beanpole,https://t1.daumcdn.net/cfile/tistory/02482D495...
10,benetton,https://upload.wikimedia.org/wikipedia/commons...
11,blackyak,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...
12,buckaroo,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...


In [14]:
for row in tqdm_notebook(logo_urls.itertuples()):
    #if row.brand not in ['beanpole','chanel','diesel','jansport','lafuma']:
    #    continue
    
    fname = 'app/static/logos/' + row.shortname + '.jpg'
    if not os.path.isfile(fname):
        try:
            img = requests.get(row.logourl).content
            with open(fname, 'wb') as f:
                f.write(img)
        except:
            print(row.shortname)

suecommabonnie
helinox
billylosangeles



In [19]:
logo_urls.logourl[logo_urls.shortname=='8seconds'].values

array(['https://ko.wikipedia.org/wiki/%EC%97%90%EC%9E%87%EC%84%B8%EC%BB%A8%EC%A6%88#/media/File:8_seconds_logo.gif'],
      dtype=object)

In [12]:
url = 'https://t1tallinn.com/wp-content/uploads/2018/10/MandarinaDuck_T1.jpg'
img = requests.get(url).content
with open('test.jpg', 'wb') as f:
    f.write(img)

# DB 초기화

In [2]:
with open('tokensfreq.json', encoding='UTF-8-sig') as f:
    wordfreqs = json.load(f)
    
def get_wordfreq(bname):
    return {k:v for k,v in wordfreqs[bname].items() if v>5}
    #return [{'text':k, 'freq':v} for k,v in wordfreqs[bname].items() if v>5]

In [7]:
df = pd.read_pickle('id_dict.pkl')
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0.1, 1))
X_train_minmax = min_max_scaler.fit_transform(df)
df[:] = X_train_minmax

def get_identity(bname):
    try:
        _df = df[[bname]].reset_index().rename(columns={'index':'key', bname:'value'})
        _df.value = (_df.value*100).astype(int)
        return _df.to_dict('record')
    except:
        return None

In [8]:
db_init = []
bnames = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215').shortname.iloc[3:]
clusters = pd.read_excel('clustered.xlsx')[['bname','cluster']].set_index('bname')

for i, brand in enumerate(bnames):
    db_init.append({
        'model': 'app.brand', 
        'pk': i, 
        'fields': {
            'name': brand, 
            'logo': brand + '.jpg', 
            'cluster': str(clusters.cluster.get(brand, '')), 
            'identity': json.dumps(get_identity(brand), ensure_ascii=False), 
            'wordfreq': json.dumps(get_wordfreq(brand), ensure_ascii=False), 
            #'wordcloud': 'wc_' + brand + '.png'
        }
    })

In [9]:
with open('db_init.json', 'w') as f: #, encoding='UTF-8-sig'
    json.dump(db_init, f)#, ensure_ascii=False)

In [11]:
db_init[0];

In [12]:
sum(np.array(list(wordfreqs['nike'].values()))<5)

23337

In [13]:
len(wordfreqs['nike'])

29759

In [6]:
dict(sorted(wordfreqs['crocs'].items(), key=lambda x: x[1])[-100:])

{'jibbitz': 9,
 '버켄': 9,
 '버켄스탁': 9,
 '아동화': 9,
 '미소': 9,
 '키즈': 9,
 'manager': 9,
 '할인': 9,
 '오후': 9,
 '엄청': 9,
 '같은': 9,
 'shoes': 10,
 '으로': 10,
 '출시': 10,
 '구매': 10,
 'now': 10,
 '스타일링': 10,
 '데이': 10,
 '부츠': 10,
 '정말': 10,
 '편한': 10,
 '영상': 11,
 'come': 11,
 'as': 11,
 'you': 11,
 '레이웨지': 11,
 '70': 11,
 '슬리퍼': 11,
 '너무': 11,
 'my': 11,
 'crocskorea': 12,
 '마감일': 12,
 '근무지': 12,
 '함께': 12,
 '이시': 12,
 '좋은': 12,
 '비치샌들': 12,
 '더클래스': 12,
 'korea': 13,
 '홍보대사': 13,
 '네모습그대로가좋아': 13,
 '에서': 13,
 '아동샌들': 13,
 '선물': 13,
 'henry': 14,
 '마감': 14,
 '포토씨': 14,
 '슈즈': 14,
 '있는': 15,
 '25': 15,
 'are': 15,
 '캠페인': 15,
 '30': 15,
 '6pm': 15,
 'liveen': 15,
 '제조': 16,
 '인스타그램': 16,
 '아동': 16,
 '미국계': 17,
 '서울': 17,
 '행사': 17,
 '회사': 18,
 '이벤트': 18,
 '추가': 18,
 '미모': 19,
 '라이트라이드': 19,
 'by': 20,
 'hd포토': 20,
 '임윤아': 21,
 '할인쿠폰': 22,
 '170925': 24,
 '샌들': 25,
 '60': 25,
 '신고': 27,
 '서울근무': 27,
 '쿠폰': 28,
 '글로벌': 28,
 'comeasyoua': 29,
 're': 29,
 '20': 30,
 'asap': 30,
 '패션': 32,
 '헨리': 33,
 '여